In [163]:
import http.client
import pandas as pd
import json
pd.set_option("display.max_rows", 200)

In [164]:
import http.client
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [165]:
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[["first_name", "last_name", "position", "score", "strokes"]]

,first_name,last_name,position,score,strokes
0,Patrick,Reed,1,-9,135
1,Marc,Leishman,2,-7,137
2,Henrik,Stenson,3,-5,139
3,Jordan,Spieth,4,-4,140
4,Rory,McIlroy,4,-4,140
5,Dustin,Johnson,6,-3,141
6,Justin,Thomas,6,-3,141
7,Louis,Oosthuizen,8,-2,142
8,Charley,Hoffman,8,-2,142
9,Bubba,Watson,8,-2,142


In [166]:
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    return int(score)
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    return int(strokes)

In [167]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [168]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [169]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [170]:
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]

In [171]:
df.to_csv("masters_results.csv")

In [172]:
df

,Pool Player,Player 1,Score 1,Player 2,Score 2,Player 3,Score 3,Player 4,Score 4,Player 5,Score 5,Player 6,Score 6,Player 7,Score 7,Total,Tie Breaker
0,Dave Peterson,Rory McIlroy,-4,Jason Day,2,Patrick Reed,-9,Tony Finau,-2,Jason Dufner,6,Tiger Woods,4,Martin Kaymer,3,1008.0,278.0
1,Joe Kearney,Justin Thomas,-3,Jason Day,2,Patrick Reed,-9,Louis Oosthuizen,-2,Thomas Pieters,7,Tiger Woods,4,Russell Henley,1,1008.0,279.0
2,Sam Montgomery,Jordan Spieth,-4,Tommy Fleetwood,0,Charley Hoffman,-2,Branden Grace,2,Adam Hadwin,0,Jimmy Walker,0,Bryson DeChambeau,4,1008.0,282.0
3,Jeremy Szyba #3,Dustin Johnson,-3,Phil Mickelson,5,Patrick Reed,-9,Branden Grace,2,Adam Hadwin,0,Daniel Berger,3,Ian Poulter,5,1011.0,278.0
4,Paul Bower,Dustin Johnson,-3,Jason Day,2,Bubba Watson,-2,Daniel Berger,3,Adam Hadwin,0,Tiger Woods,4,Jimmy Walker,0,1012.0,268.0
5,Jeremy Szyba,rickie Fowler,-2,Paul Casey,5,Bubba Watson,-2,Louis Oosthuizen,-2,Jason Dufner,6,Jimmy Walker,0,Zach Johnson,0,1013.0,277.0
6,Zachary Sight,Justin Thomas,-3,Phil Mickelson,5,Patrick Reed,-9,Daniel Berger,3,Siwoo Kim,4,Tiger Woods,4,Fred Couples,2,1014.0,278.0
7,Johnny Pendleton,Hideki Matsuyama,0,Jason Day,2,Patrick Reed,-9,Louis Oosthuizen,-2,Thomas Pieters,7,Tiger Woods,4,Bryson DeChambeau,4,1014.0,280.0
8,Tristen Woods,Jon Rahm,-1,Henrik Stenson,-5,Patrick Reed,-9,Brendan Steele,7,Cameron Smith,-1,Billy Horschel,11,Adam Scott,4,1014.0,279.0
9,Chris Stine,Rory McIlroy,-4,Phil Mickelson,5,Brian Harman,3,Louis Oosthuizen,-2,Cameron Smith,-1,Tiger Woods,4,Ryan Moore,2,1015.0,280.0
